In [ ]:
!pip install EMD-signal antropy --quiet

# 🔧 Importación de librerías

En esta celda se importan todas las librerías necesarias para el desarrollo del proyecto. Estas se agrupan por funcionalidad:

- **Librerías estándar:** manejo de archivos, advertencias, arrays y dataframes.
- **Análisis estadístico y modelado ARIMA.**
- **Análisis de señales:** descomposición EEMD y cálculo de entropía por permutación.
- **Preprocesamiento y métricas.**
- **Modelado con redes neuronales LSTM usando Keras.**
- **Carga de archivos desde Google Drive (entornos Colab).**
- **Manejo de fechas y advertencias.**


In [ ]:
# ============================
# Importación de librerías
# ============================

# Librerías estándar
import os                        # Para operaciones del sistema de archivos
import pickle                   # Para serialización de objetos en disco
import warnings                 # Para controlar la visualización de advertencias
import numpy as np              # Operaciones con arrays numéricos
import pandas as pd             # Manipulación de datos tabulares
import matplotlib.pyplot as plt # Visualización de gráficos
import seaborn as sns           # Visualizaciones estadísticas avanzadas

# Librerías estadísticas
from scipy.stats import linregress                         # Regresión lineal simple
from statsmodels.tsa.arima.model import ARIMA              # Modelado ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning  # Advertencia específica de statsmodels

# Análisis de señales
from PyEMD import EEMD               # Descomposición empírica con modo ensemble
from antropy import perm_entropy    # Cálculo de entropía por permutación

# Preprocesamiento y evaluación
from sklearn.preprocessing import MinMaxScaler                 # Normalización de datos
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # Métricas de evaluación

# Modelado LSTM con Keras
from tensorflow.keras.models import Sequential, load_model    # Creación y carga de modelos
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input  # Capas para redes neuronales LSTM

# Colab y visualización interactiva
from google.colab import drive             # Acceso a Google Drive
from IPython.display import display        # Visualización mejorada en notebooks

# Manipulación de fechas
from dateutil.relativedelta import relativedelta  # Operaciones con fechas relativas

# Suprimir advertencias de convergencia de ARIMA
warnings.simplefilter("ignore", ConvergenceWarning)


# 📁 Configuración de rutas y estructura de carpetas

Esta celda monta Google Drive (en entornos como Google Colab) y configura la estructura de carpetas del proyecto:

- Se define la ruta base del proyecto (`BASE_DIR`).
- Se especifica la ubicación del archivo de datos CSV.
- Se crea un diccionario `DIRS` con rutas organizadas para almacenar:
  - Datos originales.
  - Modelos entrenados (LSTM y ARIMA).
  - Resultados estadísticos de entrenamiento y prueba.
  - Gráficas de análisis exploratorio, EEMD y evaluación de modelos.
  - Predicciones futuras.

Finalmente, se asegura que todas las carpetas existan mediante `os.makedirs(...)`.


In [ ]:
# ===============================================
# Montaje de Google Drive y configuración inicial
# ===============================================

# Montar Google Drive para acceder a archivos almacenados en la nube
drive.mount('/content/drive')

# Ruta base del proyecto (puedes modificarla según tu estructura)
BASE_DIR = "/content/drive/MyDrive/"

# Ruta al archivo CSV con los datos de precipitación
DATA_PATH = os.path.join(BASE_DIR, "datos", "descargaDhime.csv")

# Diccionario de rutas para organizar los diferentes productos del proyecto
DIRS = {
    # Carpeta con los datos originales
    "datos": f"{BASE_DIR}/datos",

    # Modelos entrenados
    "lstm_models": f"{BASE_DIR}/modelos/lstm",         # Modelos LSTM por estación y componente
    "arima_models": f"{BASE_DIR}/modelos/arima",       # Modelos ARIMA por estación y componente

    # Resultados estadísticos de entrenamiento y prueba
    "stats_train": f"{BASE_DIR}/estadisticas/eemd/train",
    "stats_test": f"{BASE_DIR}/estadisticas/eemd/test",

    # Gráficas del análisis exploratorio y EEMD
    "graficos_precipitacion": f"{BASE_DIR}/graficas/precipitacion",
    "graficos_eda": f"{BASE_DIR}/graficas/exploratorio",
    "graficos_eemd": f"{BASE_DIR}/graficas/eemd",

    # Gráficas de evaluación en entrenamiento
    "graficos_train_mse": f"{BASE_DIR}/graficas/resultados_modelo/train/curvas_mse",
    "graficos_train_dispersion": f"{BASE_DIR}/graficas/resultados_modelo/train/dispersion",
    "graficos_train_reconstruccion": f"{BASE_DIR}/graficas/resultados_modelo/train/reconstruccion",

    # Gráficas de evaluación en validación (test)
    "graficos_test_dispersion": f"{BASE_DIR}/graficas/resultados_modelo/test/dispersion",
    "graficos_test_reconstruccion": f"{BASE_DIR}/graficas/resultados_modelo/test/reconstruccion",

    # Predicciones a futuro generadas por estación
    "predicciones_futuras": f"{BASE_DIR}/predicciones/futuro"
}

# Crear todas las carpetas necesarias si aún no existen
for ruta in DIRS.values():
    os.makedirs(ruta, exist_ok=True)


# 🧾 Carga de datos

Se define una función `cargar_datos(path)` que:

- Lee un archivo CSV desde la ruta especificada.
- Convierte la columna `'Fecha'` al tipo `datetime`, manejando errores silenciosamente.
- Devuelve un `DataFrame` con los datos listos para ser analizados.

Luego, se utiliza esta función para cargar los datos de precipitación almacenados en la ruta `DATA_PATH`.


In [ ]:
# ===================================
# Función para cargar y procesar datos
# ===================================

def cargar_datos(path):
    """
    Carga un archivo CSV y convierte la columna 'Fecha' a formato datetime.

    Parámetros:
        path (str): Ruta al archivo CSV.

    Retorna:
        pd.DataFrame: Datos cargados y preparados.
    """
    print("📥 Cargando archivo...")
    # Lee el archivo CSV usando codificación latin-1 para evitar errores con caracteres especiales
    df = pd.read_csv(path, encoding='latin-1')

    # Convierte la columna 'Fecha' a tipo datetime, ignorando errores
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

    print(f"✅ Datos cargados: {df.shape[0]} registros")
    return df

# Carga del archivo principal de datos de precipitación
df = cargar_datos(DATA_PATH)


# 🧼 Interpolación y segmentación por estación

En esta celda se define la función `interpolar_y_separar_por_estacion(df)`, que:

- Limpia los nombres de las estaciones para unificar formatos y eliminar etiquetas.
- Interpola los valores faltantes (NaN) por estación utilizando interpolación lineal.
- Genera un `DataFrame` por estación con frecuencia mensual (`'MS'`).
- Retorna un diccionario donde cada clave es una estación y el valor es su respectivo `DataFrame` ordenado por fecha.

Esto permite trabajar de manera individual con cada serie temporal de precipitación.


In [ ]:
# =====================================================
# Función para interpolar y separar datos por estación
# =====================================================

def interpolar_y_separar_por_estacion(df):
    """
    Interpola los valores faltantes por estación y separa los datos en un diccionario.

    También realiza limpieza de nombres de estaciones.

    Parámetros:
        df (pd.DataFrame): Datos crudos con columnas 'Fecha' y 'NombreEstacion'.

    Retorna:
        dict: Diccionario con DataFrames por estación.
    """
    # Limpieza de nombres de estación eliminando etiquetas al final como [M], [A], etc.
    df["NombreEstacion"] = df["NombreEstacion"].astype(str).str.replace(r'\s*\[.*\]$', '', regex=True)

    # Reemplazos manuales para homogeneizar nombres
    reemplazos = {
        "CUCHARO EL": "EL CUCHARO",
        "MAMONAL EL HACIENDA": "HACIENDA EL MAMONAL",
        "PAVAS LAS": "LAS PAVAS",
        # Agrega más casos según sea necesario
    }
    df["NombreEstacion"] = df["NombreEstacion"].replace(reemplazos)

    estaciones = df["NombreEstacion"].dropna().unique()
    df_total = []

    for est in estaciones:
        # Filtra los datos para la estación actual
        df_est = df[df["NombreEstacion"] == est].copy()

        # Genera un rango de fechas mensuales
        fechas = pd.date_range(df_est['Fecha'].min(), df_est['Fecha'].max(), freq='MS')
        df_fechas = pd.DataFrame({'Fecha': fechas})

        # Une el rango de fechas con los datos reales
        df_merge = pd.merge(df_fechas, df_est, on='Fecha', how='left')

        # Interpola columnas numéricas que tengan valores faltantes
        for col in df_merge.columns:
            if col != "Fecha" and pd.api.types.is_numeric_dtype(df_merge[col]):
                df_merge[col] = df_merge[col].interpolate(method='linear')

        # Rellena nombre de estación si quedó en NaN
        df_merge["NombreEstacion"].fillna(est, inplace=True)

        # Agrega el DataFrame procesado al conjunto total
        df_total.append(df_merge)

    # Concatenación de todos los registros procesados
    df_final = pd.concat(df_total, ignore_index=True)

    # Crea un diccionario por estación con los datos ordenados por fecha
    return {
        est: df_final[df_final["NombreEstacion"] == est].copy().sort_values("Fecha").reset_index(drop=True)
        for est in estaciones
    }

# Aplicar función a los datos cargados
df_estaciones = interpolar_y_separar_por_estacion(df)
print("✅ Estaciones procesadas:", sorted(df_estaciones.keys()))


# 📊 Visualización de la precipitación por estación

Esta celda define la función `graficar_precipitacion(df_estaciones)`, la cual:

- Genera gráficos mensuales y anuales de precipitación por estación.
- Aplica regresión lineal y medias móviles para observar tendencias a largo plazo.
- Guarda los gráficos generados en la carpeta `graficos_precipitacion`.

Cada estación tendrá:
- Un gráfico mensual con:
  - Precipitación total por mes.
  - Media móvil de 12 meses.
  - Línea de tendencia (regresión lineal).
- Un gráfico anual con:
  - Suma anual de precipitación.
  - Media móvil de 7 años.
  - Promedio histórico y línea de tendencia.

In [ ]:

# ===================================================
# Función para graficar precipitación mensual/anual
# ===================================================

def graficar_precipitacion(df_estaciones):
    """
    Genera gráficos mensuales y anuales por estación.

    Parámetros:
        df_estaciones (dict): Diccionario con DataFrames por estación.
    """
    # Asegura que el directorio de salida existe
    os.makedirs(DIRS["graficos_precipitacion"], exist_ok=True)

    for station, station_df in df_estaciones.items():
        # Limita los datos hasta el año 2024
        station_df = station_df[station_df['Fecha'].dt.year <= 2024]

        # ====================
        # Gráfico mensual
        # ====================
        monthly = station_df.set_index('Fecha')['Valor'].resample('M').sum()
        if not monthly.empty:
            rolling_12 = monthly.rolling(window=12).mean()  # Media móvil de 12 meses
            x = np.arange(len(monthly))
            slope, intercept, *_ = linregress(x, monthly)
            reg_line = intercept + slope * x

            fig, ax = plt.subplots(figsize=(10, 5))
            ax.plot(monthly.values, color='blue', label='Mensual')
            ax.plot(rolling_12.values, '--', color='orange', label='Media móvil (12m)')
            ax.plot(reg_line, color='black', label='Regresión lineal')
            ax.set_title(f"Precipitación mensual - {station}")
            ax.set_xlabel("Mes")
            ax.set_ylabel("Precipitación (mm)")
            ax.legend()
            ax.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()

            filename = station.replace('/', '_').replace(' ', '_') + '_mensual.png'
            plt.savefig(os.path.join(DIRS["graficos_precipitacion"], filename), dpi=300)
            plt.show()
            plt.close(fig)

        # ====================
        # Gráfico anual
        # ====================
        annual = station_df.set_index('Fecha')['Valor'].resample('Y').sum()
        if not annual.empty:
            rolling_7 = annual.rolling(window=7).mean()  # Media móvil de 7 años
            x_years = np.arange(len(annual))
            slope_a, intercept_a, *_ = linregress(x_years, annual)
            reg_line_a = intercept_a + slope_a * x_years
            avg_line = [annual.mean()] * len(annual)

            fig, ax = plt.subplots(figsize=(10, 5))
            ax.plot(annual.values, color='blue', label='Anual')
            ax.plot(rolling_7.values, '--', color='orange', label='Media móvil (7a)')
            ax.plot(reg_line_a, color='black', label='Regresión lineal')
            ax.plot(avg_line, color='red', label='Promedio histórico')
            ax.set_title(f"Precipitación anual - {station}")
            ax.set_xlabel("Año")
            ax.set_ylabel("Precipitación (mm)")
            ax.legend()
            ax.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()

            filename = station.replace('/', '_').replace(' ', '_') + '_anual.png'
            plt.savefig(os.path.join(DIRS["graficos_precipitacion"], filename), dpi=300)
            plt.show()
            plt.close(fig)

# Ejecutar la función para graficar todas las estaciones
graficar_precipitacion(df_estaciones)


# 🌀 Descomposición EEMD y partición de datos

Esta celda define la función `aplicar_eemd(df_estaciones)`, que:

- Aplica la técnica EEMD (Empirical Ensemble Mode Decomposition) a la serie de precipitación de cada estación.
- Divide los resultados en conjuntos de entrenamiento y prueba utilizando como fecha de corte el 1 de enero de 2020.
- Almacena las siguientes estructuras por estación:
  - `IMFs`: componentes intrínsecos (modo).
  - `serie`: serie original completa.
  - `IMFs_train`, `IMFs_test`: componentes para entrenamiento y prueba.
  - `serie_train`, `serie_test`: serie original dividida.

Además, genera gráficos con los IMFs por estación y marca visualmente la fecha de partición.


In [ ]:
# =====================================================
# Aplicar EEMD a todas las estaciones y segmentar datos
# =====================================================

def aplicar_eemd(df_estaciones):
    """
    Aplica EEMD a cada estación y divide en conjuntos de entrenamiento/prueba.

    Parámetros:
        df_estaciones (dict): Diccionario de DataFrames por estación.

    Retorna:
        dict: Resultados por estación, incluyendo IMFs, fechas y series divididas.
    """
    imfs_dict = {}

    for est, df in df_estaciones.items():
        # Agrega la serie mensual, rellenando valores faltantes con 0
        serie = df.set_index('Fecha')['Valor'].resample('MS').sum().fillna(0)
        if len(serie) == 0:
            continue

        # Aplica EEMD
        eemd = EEMD()
        IMFs = eemd.eemd(serie.values)
        fechas = serie.index

        # Define el índice de corte para dividir en train/test (enero 2020)
        split_idx = np.argmin(np.abs(fechas - pd.Timestamp("2020-01-01")))

        # Almacena los resultados en el diccionario
        imfs_dict[est] = {
            'IMFs': IMFs,
            'serie': serie.values,
            'fechas': fechas,
            'IMFs_train': IMFs[:, :split_idx],
            'IMFs_test': IMFs[:, split_idx:],
            'serie_train': serie.values[:split_idx],
            'serie_test': serie.values[split_idx:]
        }

        # ============
        # Gráfico EEMD
        # ============
        fig, axs = plt.subplots(IMFs.shape[0], 1, figsize=(12, 2 * IMFs.shape[0]), sharex=True)
        for i, ax in enumerate(axs):
            ax.plot(fechas, IMFs[i], color='blue')
            ax.axvline(pd.Timestamp("2020-01-01"), color='red', linestyle='--')
            ax.set_ylabel(f'IMF {i+1}')
            ax.grid(True)

        fig.suptitle(f"EEMD - {est}", fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(DIRS["graficos_eemd"], f"{est.replace('/','_')}_EEMD.png"))
        plt.show()
        plt.close()

    return imfs_dict

# Aplicar EEMD a las estaciones y guardar resultados
imfs_dict = aplicar_eemd(df_estaciones)


# 📈 Cálculo de estadísticas y entropía de IMFs

Esta celda define la función `calcular_estadisticas(...)`, que:

- Calcula estadísticas básicas (máximo, mínimo, media, desviación estándar) de:
  - Cada IMF (modo intrínseco).
  - La serie original ("Raw").
  - El residuo (diferencia entre la serie original y la suma de los IMFs).
- Calcula la entropía por permutación (Permutation Entropy, PE) normalizada para cada IMF y el residual.
- Clasifica los IMFs como de **alta** o **baja frecuencia** en función de la mediana de sus valores de entropía.
- Guarda los resultados como archivos CSV por estación en las carpetas correspondientes (`train` o `test`).
- Finalmente, ejecuta esta función para todos los conjuntos de entrenamiento y prueba.

Esto proporciona una caracterización estadística y espectral útil para la selección de componentes a modelar.


In [ ]:
# ==============================================================
# Función para calcular estadísticas y entropía por permutación
# ==============================================================

def calcular_estadisticas(IMFs, serie_original, estacion=None, modo='train'):
    """
    Calcula estadísticas y entropía por permutación de IMFs, serie original y residual.

    Parámetros:
        IMFs (np.ndarray): Matriz con los IMFs.
        serie_original (np.ndarray): Serie temporal original.
        estacion (str): Nombre de la estación (opcional para guardar).
        modo (str): 'train' o 'test' para definir ruta de guardado.

    Retorna:
        pd.DataFrame: Tabla con estadísticas y clasificación de frecuencia.
    """
    df_stats = []

    for i, imf in enumerate(IMFs):
        pe = perm_entropy(imf, normalize=True)
        df_stats.append({
            "IMF": f"IMF{i+1}",
            "Max": np.max(imf),
            "Min": np.min(imf),
            "Mean": np.mean(imf),
            "Std": np.std(imf),
            "PE": pe
        })

    # Calcula el residuo: diferencia entre la serie original y la suma de los IMFs
    residual = serie_original - IMFs.sum(axis=0)

    # Estadísticas de la serie original (sin entropía)
    df_raw = pd.DataFrame([{
        "IMF": "Raw",
        "Max": np.max(serie_original),
        "Min": np.min(serie_original),
        "Mean": np.mean(serie_original),
        "Std": np.std(serie_original),
        "PE": None
    }])

    # Estadísticas del residuo (incluye entropía)
    df_residual = pd.DataFrame([{
        "IMF": "Residual",
        "Max": np.max(residual),
        "Min": np.min(residual),
        "Mean": np.mean(residual),
        "Std": np.std(residual),
        "PE": perm_entropy(residual, normalize=True)
    }])

    # Combina todas las estadísticas en un único DataFrame
    df_stats = pd.concat([df_raw, pd.DataFrame(df_stats), df_residual], ignore_index=True)

    # Clasificación de IMFs por frecuencia según la mediana de entropía
    pe_values = df_stats[df_stats["IMF"].str.contains("IMF")]["PE"]
    threshold = pe_values.median()
    df_stats["Freq"] = df_stats["PE"].apply(
        lambda x: "High" if pd.notnull(x) and x >= threshold else ("Low" if pd.notnull(x) else None)
    )

    # Guardado en CSV si se especifica una estación
    if estacion:
        carpeta = DIRS["stats_train"] if modo == "train" else DIRS["stats_test"]
        archivo = f"{estacion.replace('/', '_')}.csv"
        df_stats.to_csv(os.path.join(carpeta, archivo), index=False)
        print(f"📄 Estadísticas guardadas: {archivo}")

    return df_stats

# ====================================================
# Cálculo de estadísticas para todas las estaciones
# ====================================================

stats_dict_train = {}
stats_dict_test = {}

for est, info in imfs_dict.items():
    stats_dict_train[est] = calcular_estadisticas(
        info['IMFs_train'], info['serie_train'], estacion=est, modo='train'
    )
    stats_dict_test[est] = calcular_estadisticas(
        info['IMFs_test'], info['serie_test'], estacion=est, modo='test'
    )


# 🧪 Preparación de datos para LSTM

Se define la función `preparar_serie_lstm(...)`, encargada de transformar una serie temporal en un conjunto de muestras de entrenamiento para una red LSTM.

Esta transformación se realiza utilizando una ventana deslizante:

- Entrada `X`: subseries de longitud `n_steps`.
- Salida `y`: el siguiente valor en la serie después de cada subserie.

Esto permite que la red LSTM aprenda patrones secuenciales y realice predicciones basadas en contextos anteriores.


In [ ]:
# =====================================================
# Preparación de datos para entrenamiento de LSTM
# =====================================================

def preparar_serie_lstm(serie, n_steps=10):
    """
    Genera muestras (X, y) para redes LSTM a partir de una serie temporal.

    Parámetros:
        serie (np.ndarray): Serie normalizada con forma (n, 1) o (n,).
        n_steps (int): Número de pasos de entrada (ventana temporal).

    Retorna:
        Tuple[np.ndarray, np.ndarray]: Arrays con muestras X y etiquetas y.
    """
    X, y = [], []

    # Crea secuencias deslizantes de longitud n_steps
    for i in range(len(serie) - n_steps):
        X.append(serie[i:i + n_steps])  # Subserie de entrada
        y.append(serie[i + n_steps])    # Valor a predecir

    return np.array(X), np.array(y)


# 🤖 Entrenamiento de modelos ARIMA y LSTM por frecuencia

Esta celda define la función `entrenar_modelos(...)`, que implementa el enfoque híbrido:

- Para cada estación:
  - Clasifica los IMFs en **alta** o **baja frecuencia** (según entropía).
  - **IMFs de baja frecuencia**: se modelan con **ARIMA** (orden fijo (1,0,0)).
  - **IMFs de alta frecuencia**: se modelan con **LSTM** usando una arquitectura secuencial.
  - **Componente residual** (serie original menos suma de IMFs): se puede modelar opcionalmente con ARIMA.
- Se guarda:
  - Cada modelo ARIMA (por IMF y residual) como `.pkl`.
  - Cada modelo LSTM entrenado como `.keras`.
  - El historial de entrenamiento de LSTM en el diccionario `histories`.

Se retorna:
- `pred_arima`: predicción acumulada de los componentes modelados con ARIMA.
- `pred_lstm`: predicción acumulada de los componentes modelados con LSTM.
- `histories`: métricas de pérdida por época para cada LSTM.


In [ ]:
# ========================================================
# Entrenamiento de modelos híbridos ARIMA y LSTM por IMF
# ========================================================

def entrenar_modelos(imfs_dict, stats_dict, n_steps=10, usar_residual=True):
    """
    Entrena modelos ARIMA (frecuencia baja y residual) y LSTM (frecuencia alta).

    Parámetros:
        imfs_dict (dict): Diccionario con IMFs y series por estación.
        stats_dict (dict): Tabla con clasificación de frecuencia por IMF.
        n_steps (int): Tamaño de la ventana para LSTM.
        usar_residual (bool): Si se incluye modelo ARIMA sobre el residual.

    Retorna:
        Tuple[dict, dict, dict]: Predicciones ARIMA, LSTM y historiales LSTM.
    """
    pred_arima, pred_lstm, histories = {}, {}, {}

    for est, datos in imfs_dict.items():
        print(f"🔧 Entrenando modelos para: {est}")
        nombre_archivo = est.replace('/', '_').replace(' ', '_')

        imfs_train = datos['IMFs_train']
        serie_train = datos['serie_train']
        residual = serie_train - imfs_train.sum(axis=0)
        df_stats = stats_dict[est]

        # Inicializar predicciones
        pred_arima[est] = np.zeros(len(serie_train))
        pred_lstm[est] = np.zeros(len(serie_train))

        # Clasifica los IMFs según su frecuencia
        altas, bajas = [], []
        for _, row in df_stats.iterrows():
            if isinstance(row["IMF"], str) and "IMF" in row["IMF"]:
                idx = int(row["IMF"].replace("IMF", "")) - 1
                (altas if row["Freq"] == "High" else bajas).append(idx)

        # ========================
        # Modelado ARIMA (bajas)
        # ========================
        for i in bajas:
            try:
                modelo = ARIMA(imfs_train[i], order=(1, 0, 0)).fit()
                pred = modelo.predict(start=1, end=len(imfs_train[i]) - 1)
                pred = np.insert(pred, 0, 0)  # Alineación con el inicio
                pred_arima[est] += pred

                # Guardar modelo ARIMA en disco
                with open(f"{DIRS['arima_models']}/{nombre_archivo}_imf{i}_arima.pkl", "wb") as f:
                    pickle.dump(modelo, f)
            except Exception as e:
                print(f"⚠️ ARIMA IMF{i+1} - {e}")

        # ========================
        # Modelado LSTM (altas)
        # ========================
        for i in altas:
            serie = imfs_train[i].reshape(-1, 1)
            scaler = MinMaxScaler()
            serie_scaled = scaler.fit_transform(serie)

            X, y = preparar_serie_lstm(serie_scaled, n_steps)
            if len(X) == 0:
                continue

            X = X.reshape((X.shape[0], X.shape[1], 1))

            # Definición del modelo LSTM
            model = Sequential([
                Input(shape=(n_steps, 1)),
                LSTM(128),
                Dropout(0.2),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse')

            # Entrenamiento silencioso
            hist = model.fit(X, y, epochs=100, verbose=0)
            histories[f"{est}_imf{i}"] = hist.history

            # Guardar modelo
            model.save(f"{DIRS['lstm_models']}/{nombre_archivo}_imf{i}_lstm.keras")

            # Predicciones sobre entrenamiento
            y_pred = model.predict(X, verbose=0)
            y_pred_inv = scaler.inverse_transform(y_pred).flatten()

            pred_temp = np.zeros(len(serie_train))
            pred_temp[n_steps:] = y_pred_inv  # Ajuste por ventana inicial
            pred_lstm[est] += pred_temp

        # ========================
        # ARIMA para residual
        # ========================
        if usar_residual:
            try:
                modelo = ARIMA(residual, order=(1, 0, 0)).fit()
                pred = modelo.predict(start=1, end=len(residual) - 1)
                pred = np.insert(pred, 0, 0)
                pred_arima[est] += pred

                with open(f"{DIRS['arima_models']}/{nombre_archivo}_residual_arima.pkl", "wb") as f:
                    pickle.dump(modelo, f)
            except Exception as e:
                print(f"⚠️ Residual ARIMA - {e}")

    return pred_arima, pred_lstm, histories

# ====================================
# Ejecutar entrenamiento de los modelos
# ====================================

pred_arima, pred_lstm, histories = entrenar_modelos(
    imfs_dict, stats_dict_train, n_steps=10
)


# 🧩 Gráficos de reconstrucción del conjunto de entrenamiento

Esta celda define la función `graficar_reconstruccion(...)`, que:

- Toma las predicciones de ARIMA y LSTM para cada estación.
- Suma ambas para generar la **serie reconstruida** del entrenamiento.
- Compara la serie real con la reconstruida mediante un gráfico temporal.
- Guarda cada gráfico en la ruta `graficos_train_reconstruccion`.

Esta visualización permite evaluar visualmente qué tan bien el modelo híbrido reproduce la serie original.


In [ ]:
# ========================================================
# Gráfico de reconstrucción: Real vs. Predicción Total
# ========================================================

def graficar_reconstruccion(imfs_dict, pred_arima, pred_lstm):
    for est in imfs_dict:
        serie_real = imfs_dict[est]['serie_train']
        fechas = imfs_dict[est]['fechas'][:len(serie_real)]

        # Verifica que existan predicciones para la estación
        if est not in pred_arima or est not in pred_lstm:
            continue

        # Suma de predicciones ARIMA + LSTM
        pred_total = pred_arima[est] + pred_lstm[est]

        # Genera el gráfico
        plt.figure(figsize=(14, 5))
        plt.plot(fechas, serie_real, label="Serie Real", color='black')
        plt.plot(fechas, pred_total, '--', label="Reconstrucción", color='green')
        plt.title(f"Reconstrucción del conjunto de entrenamiento - {est}")
        plt.xlabel("Fecha")
        plt.ylabel("Precipitación (mm)")
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Guardar gráfico en la carpeta correspondiente
        ruta = os.path.join(
            DIRS["graficos_train_reconstruccion"],
            f"{est.replace('/', '_').replace(' ', '_')}_reconstruccion_train.png"
        )
        plt.savefig(ruta, dpi=300)
        plt.show()
        plt.close()


# 📈 Evaluación cuantitativa del conjunto de entrenamiento

Se define la función `evaluar_modelos(...)`, que evalúa el desempeño de los modelos entrenados para cada estación usando las siguientes métricas:

- **RMSE**: raíz del error cuadrático medio.
- **MAE**: error absoluto medio.
- **MSE**: error cuadrático medio.
- **R²**: coeficiente de determinación.

Se calcula el rendimiento por separado para:
- Predicción parcial por ARIMA.
- Predicción parcial por LSTM.
- Predicción total (ARIMA + LSTM).

Los resultados se imprimen por estación y ayudan a comparar la contribución relativa de cada modelo.


In [ ]:
# ============================================================
# Evaluación numérica del desempeño de los modelos (entrenamiento)
# ============================================================

def evaluar_modelos(imfs_dict, pred_arima, pred_lstm):
    print("📊 Evaluación en conjunto de entrenamiento")

    for est in imfs_dict:
        # Serie real observada
        real = imfs_dict[est]['serie_train']

        # Predicciones parciales
        pred_a = pred_arima.get(est)
        pred_l = pred_lstm.get(est)

        if pred_a is None or pred_l is None:
            continue

        # Predicción total (modelo híbrido)
        pred_total = pred_a + pred_l

        print(f"\n📍 Estación: {est}")
        for nombre, pred in [("ARIMA", pred_a), ("LSTM", pred_l), ("Total", pred_total)]:
            rmse = np.sqrt(mean_squared_error(real, pred))
            mae = mean_absolute_error(real, pred)
            mse = mean_squared_error(real, pred)
            r2 = r2_score(real, pred)
            print(f"  {nombre:6s} → RMSE: {rmse:.2f}, MAE: {mae:.2f}, MSE: {mse:.2f}, R²: {r2:.2f}")


# 📉 Visualización de curvas de aprendizaje (MSE por época)

Esta celda define la función `graficar_curvas_aprendizaje_mse(...)`, que:

- Recorre el historial de entrenamiento de los modelos LSTM para cada IMF de cada estación.
- Grafica la evolución del error cuadrático medio (MSE) a lo largo de las épocas.
- Guarda una imagen por estación en la ruta:  
  `graficas/resultados_modelo/train/curvas_mse/`.

Estas curvas permiten detectar:
- Convergencia adecuada del modelo.
- Posible sobreajuste (overfitting).
- Desempeño por componente IMF.


In [ ]:
# ===========================================================
# Gráficas de curvas de aprendizaje de LSTM (MSE por épocas)
# ===========================================================

def graficar_curvas_aprendizaje_mse(histories, base_dir=BASE_DIR):
    # Directorio de salida para las curvas
    output_dir = os.path.join(base_dir, "graficas", "resultados_modelo", "train", "curvas_mse")
    os.makedirs(output_dir, exist_ok=True)

    # Estaciones con historiales disponibles (ignorando residuales)
    estaciones = sorted(set(k.split('_')[0] for k in histories if "residual" not in k))

    for estacion in estaciones:
        # Obtiene claves IMF asociadas a la estación
        claves = [k for k in histories if k.startswith(estacion) and "residual" not in k]
        if not claves:
            print(f"⚠️ No hay historiales para estación: {estacion}")
            continue

        # Plot de la curva de entrenamiento por IMF
        plt.figure(figsize=(10, 5))
        for clave in claves:
            historial = histories[clave]
            componente = clave.split('_')[-1]  # Ej: imf2
            plt.plot(historial['loss'], label=f"{componente}")

        plt.title(f"Curva de aprendizaje - {estacion}")
        plt.xlabel("Épocas")
        plt.ylabel("MSE")
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.legend(title="IMF")
        plt.tight_layout()

        # Guardar gráfico
        nombre = estacion.replace("/", "_").replace(" ", "_") + "_mse_epochs_train.png"
        plt.savefig(os.path.join(output_dir, nombre), dpi=300)
        plt.show()
        plt.close()

    print(f"✅ Curvas de aprendizaje guardadas en: {output_dir}")


# 📌 Dispersión: Valor real vs. predicho (conjunto de entrenamiento)

Esta celda define la función `graficar_dispersión_entrenamiento(...)`, que:

- Calcula la **reconstrucción total** (ARIMA + LSTM) para cada estación.
- Genera un gráfico de dispersión que compara los valores reales y los predichos.
- Dibuja la línea de identidad (roja discontinua), que representa una predicción perfecta.
- Guarda cada gráfico en la carpeta:  
  `graficas/resultados_modelo/train/dispersion`.

Esta visualización ayuda a:
- Evaluar el **alineamiento entre la predicción y los datos reales**.
- Identificar **sesgos sistemáticos o errores de predicción**.


In [ ]:
# ============================================================
# Gráfica de dispersión Real vs Predicción (entrenamiento)
# ============================================================

def graficar_dispersión_entrenamiento(imfs_dict, pred_arima, pred_lstm, base_dir=BASE_DIR):
    # Reconstrucción híbrida (ARIMA + LSTM)
    reconstruccion_total_train = {
        est: pred_arima[est] + pred_lstm[est]
        for est in pred_arima
    }

    # Directorio de salida
    output_dir = os.path.join(base_dir, "graficas", "resultados_modelo", "train", "dispersion")
    os.makedirs(output_dir, exist_ok=True)

    for estacion, predicha in reconstruccion_total_train.items():
        real = imfs_dict[estacion]['serie_train']

        plt.figure(figsize=(7, 7))
        sns.scatterplot(
            x=real, y=predicha,
            alpha=0.6, color='dodgerblue',
            edgecolor='k', s=60
        )

        # Línea de identidad (predicción perfecta)
        min_val = min(np.min(real), np.min(predicha))
        max_val = max(np.max(real), np.max(predicha))
        plt.plot([min_val, max_val], [min_val, max_val], '--', color='red', label='Línea perfecta')

        plt.xlabel("Valor real")
        plt.ylabel("Valor predicho")
        plt.title(f"{estacion} — Dispersión Real vs Predicción")
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.legend()
        plt.tight_layout()

        # Guardar figura
        nombre_archivo = estacion.replace("/", "_").replace(" ", "_") + "_dispersion.png"
        ruta_salida = os.path.join(output_dir, nombre_archivo)
        plt.savefig(ruta_salida, dpi=300)
        plt.show()
        plt.close()

    print(f"✅ Gráficas de dispersión guardadas en: {output_dir}")


# 🧪 Evaluación del conjunto de entrenamiento

En esta celda se ejecutan las funciones de evaluación y visualización para el conjunto de entrenamiento. Se incluyen:

1. **Reconstrucción temporal**: comparación entre la serie real y la reconstruida (ARIMA + LSTM).
2. **Evaluación cuantitativa**: cálculo de métricas de error (RMSE, MAE, MSE, R²) por componente y total.
3. **Curvas de aprendizaje**: evolución del error MSE durante el entrenamiento de cada LSTM por IMF.
4. **Gráficos de dispersión**: comparación punto a punto entre valores reales y predichos para cada estación.

Estas visualizaciones y métricas son esenciales para validar la calidad del modelo híbrido en el conjunto de entrenamiento.


In [ ]:
# =============================================
# Evaluación del desempeño en entrenamiento
# =============================================

# 1. Reconstrucción visual por estación
graficar_reconstruccion(imfs_dict, pred_arima, pred_lstm)

# 2. Métricas cuantitativas (RMSE, MAE, MSE, R²)
evaluar_modelos(imfs_dict, pred_arima, pred_lstm)

# 3. Curvas de aprendizaje (MSE vs. épocas)
graficar_curvas_aprendizaje_mse(histories)

# 4. Dispersión Real vs Predicción
graficar_dispersión_entrenamiento(imfs_dict, pred_arima, pred_lstm)


# 🧪 Evaluación del modelo híbrido en el conjunto de prueba

Esta celda define la función `evaluar_modelos_test(...)`, que:

- Aplica los modelos ya entrenados (ARIMA y LSTM) a los datos de prueba (`IMFs_test` y `residual_test`) de cada estación.
- Para cada estación:
  - Carga los modelos ARIMA desde disco (`.pkl`).
  - Carga los modelos LSTM desde disco (`.keras`) y genera predicciones a partir de las subseries del test.
  - Calcula la predicción total como la suma de ARIMA + LSTM.
- Evalúa el desempeño utilizando las métricas:
  - **RMSE**: raíz del error cuadrático medio.
  - **MAE**: error absoluto medio.
  - **MSE**: error cuadrático medio.
  - **R²**: coeficiente de determinación.

Este procedimiento permite validar la capacidad de generalización del modelo híbrido sobre datos no vistos.


In [ ]:
# ============================================================
# Evaluación del modelo híbrido en el conjunto de prueba
# ============================================================

def evaluar_modelos_test(imfs_dict, stats_dict_test, n_steps=10):
    """
    Evalúa el modelo híbrido en el conjunto de prueba.

    Parámetros:
        imfs_dict (dict): Diccionario con datos de prueba por estación.
        stats_dict_test (dict): Clasificación de IMFs para el test.
        n_steps (int): Longitud de ventana para LSTM.
    """
    print("📊 Evaluación en conjunto de prueba")

    for est, datos in imfs_dict.items():
        print(f"\n📍 Estación: {est}")
        nombre_archivo = est.replace('/', '_').replace(' ', '_')

        imfs_test = datos['IMFs_test']
        serie_test = datos['serie_test']
        residual_test = serie_test - imfs_test.sum(axis=0)
        df_stats = stats_dict_test[est]

        # Inicializar predicciones vacías
        pred_arima = np.zeros(len(serie_test))
        pred_lstm = np.zeros(len(serie_test))

        # Clasifica los IMFs según frecuencia
        altas, bajas = [], []
        for _, row in df_stats.iterrows():
            if "IMF" in str(row["IMF"]):
                idx = int(row["IMF"].replace("IMF", "")) - 1
                (altas if row["Freq"] == "High" else bajas).append(idx)

        # ========================
        # ARIMA - IMFs de baja frecuencia
        # ========================
        for i in bajas:
            try:
                model_path = os.path.join(DIRS["arima_models"], f"{nombre_archivo}_imf{i}_arima.pkl")
                with open(model_path, "rb") as f:
                    modelo = pickle.load(f)
                pred = modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + len(imfs_test[i]) - 1)
                pred_arima += pred
            except Exception as e:
                print(f"⚠️ ARIMA IMF{i+1} - {e}")

        # ========================
        # LSTM - IMFs de alta frecuencia
        # ========================
        for i in altas:
            try:
                serie = imfs_test[i].reshape(-1, 1)
                scaler = MinMaxScaler()
                serie_scaled = scaler.fit_transform(serie)
                X_test, _ = preparar_serie_lstm(serie_scaled, n_steps)
                if len(X_test) == 0:
                    continue
                X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

                model_path = os.path.join(DIRS["lstm_models"], f"{nombre_archivo}_imf{i}_lstm.keras")
                model = load_model(model_path)
                y_pred = model.predict(X_test, verbose=0)
                y_pred_inv = scaler.inverse_transform(y_pred).flatten()

                pred_temp = np.zeros(len(serie_test))
                pred_temp[n_steps:] = y_pred_inv
                pred_lstm += pred_temp
            except Exception as e:
                print(f"⚠️ LSTM IMF{i+1} - {e}")

        # ========================
        # ARIMA - Residual
        # ========================
        try:
            model_path = os.path.join(DIRS["arima_models"], f"{nombre_archivo}_residual_arima.pkl")
            with open(model_path, "rb") as f:
                modelo = pickle.load(f)
            pred = modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + len(residual_test) - 1)
            pred_arima += pred
        except Exception as e:
            print(f"⚠️ Residual ARIMA - {e}")

        # ========================
        # Evaluación final
        # ========================
        pred_total = pred_arima + pred_lstm

        for nombre, pred in [("ARIMA", pred_arima), ("LSTM", pred_lstm), ("Total", pred_total)]:
            rmse = np.sqrt(mean_squared_error(serie_test, pred))
            mae = mean_absolute_error(serie_test, pred)
            mse = mean_squared_error(serie_test, pred)
            r2 = r2_score(serie_test, pred)
            print(f"  {nombre:6s} → RMSE: {rmse:.2f}, MAE: {mae:.2f}, MSE: {mse:.2f}, R²: {r2:.2f}")


# 🖼️ Reconstrucción del conjunto de prueba

Se define la función `graficar_reconstruccion_test(...)`, que:

- Aplica los modelos entrenados (ARIMA y LSTM) a los datos de prueba (`IMFs_test`, `residual_test`).
- Calcula la predicción total por estación.
- Grafica la serie real del conjunto de prueba vs. la serie reconstruida.
- Guarda cada gráfico en la ruta:
  `graficas/resultados_modelo/test/reconstruccion`.

Estas gráficas permiten validar visualmente la calidad de las predicciones sobre datos no vistos.


In [ ]:
# ============================================================
# Reconstrucción visual del conjunto de prueba
# ============================================================

def graficar_reconstruccion_test(imfs_dict, stats_dict_test, n_steps=10):
    print("🖼️ Generando gráficas de reconstrucción (test)...")

    for est, datos in imfs_dict.items():
        nombre_archivo = est.replace('/', '_').replace(' ', '_')

        imfs_test = datos['IMFs_test']
        serie_test = datos['serie_test']
        fechas_test = datos['fechas'][-len(serie_test):]
        df_stats = stats_dict_test[est]
        residual_test = serie_test - imfs_test.sum(axis=0)

        pred_arima = np.zeros(len(serie_test))
        pred_lstm = np.zeros(len(serie_test))

        # Clasificación de IMFs por frecuencia
        altas, bajas = [], []
        for _, row in df_stats.iterrows():
            if "IMF" in str(row["IMF"]):
                idx = int(row["IMF"].replace("IMF", "")) - 1
                (altas if row["Freq"] == "High" else bajas).append(idx)

        # Predicción con modelos ARIMA (IMFs de baja frecuencia)
        for i in bajas:
            try:
                with open(os.path.join(DIRS["arima_models"], f"{nombre_archivo}_imf{i}_arima.pkl"), "rb") as f:
                    modelo = pickle.load(f)
                pred = modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + len(imfs_test[i]) - 1)
                pred_arima += pred
            except:
                continue

        # Predicción con modelos LSTM (IMFs de alta frecuencia)
        for i in altas:
            try:
                serie = imfs_test[i].reshape(-1, 1)
                scaler = MinMaxScaler()
                serie_scaled = scaler.fit_transform(serie)
                X_test, _ = preparar_serie_lstm(serie_scaled, n_steps)
                if len(X_test) == 0:
                    continue
                X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

                model = load_model(os.path.join(DIRS["lstm_models"], f"{nombre_archivo}_imf{i}_lstm.keras"))
                y_pred = model.predict(X_test, verbose=0)
                y_pred_inv = scaler.inverse_transform(y_pred).flatten()

                pred_temp = np.zeros(len(serie_test))
                pred_temp[n_steps:] = y_pred_inv
                pred_lstm += pred_temp
            except:
                continue

        # Predicción con ARIMA para el residual
        try:
            with open(os.path.join(DIRS["arima_models"], f"{nombre_archivo}_residual_arima.pkl"), "rb") as f:
                modelo = pickle.load(f)
            pred = modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + len(residual_test) - 1)
            pred_arima += pred
        except:
            pass

        # Reconstrucción total
        pred_total = pred_arima + pred_lstm

        # Gráfico de reconstrucción
        plt.figure(figsize=(14, 5))
        plt.plot(fechas_test, serie_test, label="Serie Real (Test)", color='black')
        plt.plot(fechas_test, pred_total, '--', label="Reconstrucción (Test)", color='green')
        plt.title(f"Reconstrucción del conjunto de prueba - {est}")
        plt.xlabel("Fecha")
        plt.ylabel("Precipitación (mm)")
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Guardado
        ruta = os.path.join(
            DIRS["graficos_test_reconstruccion"],
            f"{nombre_archivo}_reconstruccion_test.png"
        )
        os.makedirs(os.path.dirname(ruta), exist_ok=True)
        plt.savefig(ruta, dpi=300)
        plt.show()
        plt.close()


# 📌 Dispersión: Valor real vs. predicho (conjunto de prueba)

Esta celda define la función `graficar_dispersión_test(...)`, que:

- Suma las predicciones generadas por los modelos ARIMA y LSTM sobre los `IMFs_test` y el residual.
- Compara gráficamente los valores reales y predichos de precipitación mensual.
- Dibuja la **línea de identidad** (línea perfecta en rojo), que representa una predicción ideal.
- Guarda cada gráfico en:
  `graficas/resultados_modelo/test/dispersion`.

Estas gráficas permiten validar visualmente el ajuste del modelo híbrido sobre datos de prueba (no vistos durante el entrenamiento).


In [ ]:
# =============================================================
# Gráfica de dispersión Real vs Predicción (conjunto de prueba)
# =============================================================

def graficar_dispersión_test(imfs_dict, stats_dict_test, n_steps=10):
    print("🔍 Generando gráficas de dispersión (test)...")

    output_dir = os.path.join(BASE_DIR, "graficas", "resultados_modelo", "test", "dispersion")
    os.makedirs(output_dir, exist_ok=True)

    for est, datos in imfs_dict.items():
        nombre_archivo = est.replace('/', '_').replace(' ', '_')

        imfs_test = datos['IMFs_test']
        serie_test = datos['serie_test']
        df_stats = stats_dict_test[est]
        residual_test = serie_test - imfs_test.sum(axis=0)

        pred_arima = np.zeros(len(serie_test))
        pred_lstm = np.zeros(len(serie_test))

        # Clasificación de IMFs por entropía
        altas, bajas = [], []
        for _, row in df_stats.iterrows():
            if "IMF" in str(row["IMF"]):
                idx = int(row["IMF"].replace("IMF", "")) - 1
                (altas if row["Freq"] == "High" else bajas).append(idx)

        # Predicción ARIMA para IMFs de baja frecuencia
        for i in bajas:
            try:
                with open(os.path.join(DIRS["arima_models"], f"{nombre_archivo}_imf{i}_arima.pkl"), "rb") as f:
                    modelo = pickle.load(f)
                pred_arima += modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + len(imfs_test[i]) - 1)
            except:
                continue

        # Predicción LSTM para IMFs de alta frecuencia
        for i in altas:
            try:
                serie = imfs_test[i].reshape(-1, 1)
                scaler = MinMaxScaler()
                serie_scaled = scaler.fit_transform(serie)
                X_test, _ = preparar_serie_lstm(serie_scaled, n_steps)
                if len(X_test) == 0:
                    continue
                X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
                model = load_model(os.path.join(DIRS["lstm_models"], f"{nombre_archivo}_imf{i}_lstm.keras"))
                y_pred = model.predict(X_test, verbose=0)
                y_pred_inv = scaler.inverse_transform(y_pred).flatten()
                pred_temp = np.zeros(len(serie_test))
                pred_temp[n_steps:] = y_pred_inv
                pred_lstm += pred_temp
            except:
                continue

        # Predicción ARIMA para el residual
        try:
            with open(os.path.join(DIRS["arima_models"], f"{nombre_archivo}_residual_arima.pkl"), "rb") as f:
                modelo = pickle.load(f)
            pred_arima += modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + len(residual_test) - 1)
        except:
            pass

        # Suma total
        pred_total = pred_arima + pred_lstm

        # Gráfico de dispersión
        plt.figure(figsize=(7, 7))
        sns.scatterplot(x=serie_test, y=pred_total, alpha=0.6, color='dodgerblue', edgecolor='k', s=60)

        min_val = min(np.min(serie_test), np.min(pred_total))
        max_val = max(np.max(serie_test), np.max(pred_total))
        plt.plot([min_val, max_val], [min_val, max_val], '--', color='red', label='Línea perfecta')

        plt.xlabel("Valor real (test)")
        plt.ylabel("Valor predicho (test)")
        plt.title(f"{est} — Dispersión Real vs Predicción (test)")
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.legend()
        plt.tight_layout()

        plt.savefig(os.path.join(output_dir, f"{nombre_archivo}_dispersion_test.png"), dpi=300)
        plt.show()
        plt.close()

    print(f"✅ Gráficas de dispersión guardadas en: {output_dir}")


# 🧪 Evaluación del conjunto de prueba (Test)

En esta celda se ejecutan las funciones de evaluación y visualización del modelo híbrido aplicadas al conjunto de prueba. Se incluyen:

1. **Evaluación cuantitativa**: Cálculo de métricas de error (RMSE, MAE, MSE, R²) para ARIMA, LSTM y la suma de ambos.
2. **Reconstrucción temporal**: Comparación entre la serie real de prueba y la reconstrucción del modelo.
3. **Dispersión Real vs Predicción**: Visualización punto a punto que permite evaluar el alineamiento entre la predicción y los valores observados.

Estas herramientas permiten validar la capacidad de generalización del modelo sobre datos no utilizados durante el entrenamiento.


In [ ]:
# ======================================================
# Evaluación y visualización del conjunto de prueba
# ======================================================

# 1. Evaluación numérica (métricas por estación)
evaluar_modelos_test(imfs_dict, stats_dict_test, n_steps=10)

# 2. Reconstrucción gráfica de la serie (test)
graficar_reconstruccion_test(imfs_dict, stats_dict_test, n_steps=10)

# 3. Gráfica de dispersión Real vs Predicción (test)
graficar_dispersión_test(imfs_dict, stats_dict_test, n_steps=10)


# 📅 Predicción futura (proyección mensual)

Esta celda define la función `generar_predicciones_futuras(...)`, que:

- Utiliza los modelos ya entrenados (ARIMA y LSTM) para proyectar la precipitación mensual en cada estación durante los próximos `pasos` meses (por defecto: 35).
- Aplica:
  - Modelos ARIMA para los IMFs de baja frecuencia.
  - Modelos LSTM con predicción autoregresiva para los IMFs de alta frecuencia.
  - ARIMA para el componente residual.
- Suma todas las predicciones por componente para obtener la serie proyectada total.
- Genera una tabla por estación con las fechas futuras y sus valores predichos.
- Retorna un diccionario con DataFrames estructurados como:  
  `{estación: DataFrame con columnas ['Fecha', 'Prediccion']}`.

Este procedimiento permite extender las estimaciones del modelo más allá del periodo observado.


In [ ]:
# ===========================================================
# Generación de predicciones futuras (35 meses por estación)
# ===========================================================

def generar_predicciones_futuras(imfs_dict, stats_dict_train, n_steps=10, pasos=35):
    """
    Genera predicciones futuras de precipitación para cada estación.

    Parámetros:
        imfs_dict (dict): Contiene IMFs, serie completa y fechas.
        stats_dict_train (dict): Estadísticas y clasificación de IMFs.
        n_steps (int): Tamaño de la ventana para LSTM.
        pasos (int): Meses a predecir.

    Retorna:
        dict: {estación: DataFrame con columnas ['Fecha', 'Prediccion']}
    """
    predicciones_futuras = {}

    for est, datos in imfs_dict.items():
        print(f"📈 Prediciendo futuro para estación: {est}")
        nombre_archivo = est.replace('/', '_').replace(' ', '_')

        imfs_full = datos['IMFs']
        serie_full = datos['serie']
        fechas_full = datos['fechas']
        df_stats = stats_dict_train[est]

        pred_total = np.zeros(pasos)
        altas, bajas = [], []

        # Clasificación por entropía
        for _, row in df_stats.iterrows():
            if "IMF" in str(row["IMF"]):
                idx = int(row["IMF"].replace("IMF", "")) - 1
                (altas if row["Freq"] == "High" else bajas).append(idx)

        # ========================
        # Predicción con ARIMA
        # ========================
        for i in bajas:
            try:
                model_path = os.path.join(DIRS["arima_models"], f"{nombre_archivo}_imf{i}_arima.pkl")
                with open(model_path, "rb") as f:
                    modelo = pickle.load(f)
                pred = modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + pasos - 1)
                pred_total += pred
            except Exception as e:
                print(f"⚠️ ARIMA IMF{i+1} - {e}")

        # ========================
        # Predicción con LSTM
        # ========================
        for i in altas:
            try:
                serie = imfs_full[i].reshape(-1, 1)
                scaler = MinMaxScaler()
                serie_scaled = scaler.fit_transform(serie)

                # Última ventana conocida
                ultimos_valores = serie_scaled[-n_steps:]
                model_path = os.path.join(DIRS["lstm_models"], f"{nombre_archivo}_imf{i}_lstm.keras")
                model = load_model(model_path)

                pred = []
                entrada = ultimos_valores.copy()

                for _ in range(pasos):
                    X_input = entrada.reshape((1, n_steps, 1))
                    pred_scaled = model.predict(X_input, verbose=0)
                    pred.append(pred_scaled[0][0])
                    entrada = np.append(entrada[1:], pred_scaled).reshape(n_steps, 1)

                pred_inv = scaler.inverse_transform(np.array(pred).reshape(-1, 1)).flatten()
                pred_total += pred_inv
            except Exception as e:
                print(f"⚠️ LSTM IMF{i+1} - {e}")

        # ========================
        # Predicción del residual con ARIMA
        # ========================
        try:
            residual = serie_full - imfs_full.sum(axis=0)
            model_path = os.path.join(DIRS["arima_models"], f"{nombre_archivo}_residual_arima.pkl")
            with open(model_path, "rb") as f:
                modelo = pickle.load(f)
            pred = modelo.predict(start=len(modelo.model.endog), end=len(modelo.model.endog) + pasos - 1)
            pred_total += pred
        except Exception as e:
            print(f"⚠️ Residual ARIMA - {e}")

        # ========================
        # Construcción del DataFrame de resultados
        # ========================
        fecha_inicio = fechas_full[-1] + relativedelta(months=1)
        fechas_futuras = pd.date_range(fecha_inicio, periods=pasos, freq='MS')
        df_pred = pd.DataFrame({"Fecha": fechas_futuras, "Prediccion": pred_total})
        predicciones_futuras[est] = df_pred

    return predicciones_futuras


# 📊 Generación de tablas de predicción futura por estación

La función `generar_tablas_predicciones(...)`:

- Recibe:
  - `df_original`: dataset original con datos históricos reales.
  - `pred_futuras`: diccionario con predicciones generadas por estación.
- Para cada estación:
  - Convierte las fechas futuras a columnas de mes (en español) y año.
  - Crea una tabla estilo calendario (Año × Mes) con las predicciones.
  - Calcula el **promedio histórico mensual** de cada estación con base en los datos originales.
  - Añade este promedio como una fila adicional de referencia.
  - Muestra la tabla redondeada a 2 decimales.
  - Guarda la tabla en un archivo `.csv` en la ruta `predicciones_futuras`.

Estas tablas permiten una visualización clara y exportable de los resultados para informes técnicos o toma de decisiones.


In [ ]:
# ==========================================================
# Tablas anuales con predicciones mensuales por estación
# ==========================================================

def generar_tablas_predicciones(df_original, pred_futuras):
    meses_es = ['Enero','Febrero','Marzo','Abril','Mayo','Junio',
                'Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

    mapping = {
        'January': 'Enero', 'February': 'Febrero', 'March': 'Marzo',
        'April': 'Abril', 'May': 'Mayo', 'June': 'Junio',
        'July': 'Julio', 'August': 'Agosto', 'September': 'Septiembre',
        'October': 'Octubre', 'November': 'Noviembre', 'December': 'Diciembre'
    }

    for est, df_pred in pred_futuras.items():
        # Extrae año y mes en español
        df_pred["Año"] = df_pred["Fecha"].dt.year
        df_pred["Mes"] = df_pred["Fecha"].dt.month_name().map(mapping)

        # Reorganiza en tabla Año × Mes
        tabla = df_pred.pivot_table(index="Año", columns="Mes", values="Prediccion", aggfunc='sum')
        tabla = tabla[[m for m in meses_es if m in tabla.columns]]

        # Calcula promedio histórico mensual
        promedio_mensual = (
            df_original[df_original['NombreEstacion'] == est]
            .assign(Mes=lambda x: x['Fecha'].dt.month_name().map(mapping))
            .groupby("Mes")['Valor'].mean()
            .reindex(meses_es)
        )

        # Agrega el promedio histórico como fila
        tabla.loc["Promedio histórico"] = promedio_mensual.values

        # Muestra la tabla en pantalla
        display(tabla.round(2))

        # Guarda como CSV
        ruta = os.path.join(DIRS["predicciones_futuras"], f"{est.replace('/', '_')}_prediccion_futura.csv")
        tabla.to_csv(ruta)
        print(f"📁 CSV guardado: {ruta}")


# ⏩ Ejecución de predicción futura y generación de tablas

En esta celda se ejecuta:

1. `generar_predicciones_futuras(...)`: proyecta la precipitación mensual para los próximos 35 meses por estación, utilizando los modelos ARIMA y LSTM previamente entrenados.
2. `generar_tablas_predicciones(...)`: organiza los resultados en tablas tipo calendario (Año × Mes) y agrega el promedio histórico mensual como referencia. Cada tabla se guarda en formato `.csv` para su uso posterior.

Este paso final entrega un resumen cuantitativo claro de las predicciones futuras por estación.


In [ ]:
# =====================================================
# Ejecutar predicción futura (35 meses) y generar tablas
# =====================================================

pred_futuras = generar_predicciones_futuras(imfs_dict, stats_dict_train, n_steps=10, pasos=35)
generar_tablas_predicciones(df, pred_futuras)
